## <font color=blue> Lec5 协议设计</font>   
-----------------------------------------------------------

In [1]:
import socket

### 两种类型的socket

#### UDP socket

In [2]:
sock_udp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [3]:
sock_udp.sendto(b'Fake', ('127.0.0.1', 7))

4

In [4]:
sock_udp.getsockname()

('0.0.0.0', 63779)

In [5]:
data, addr = sock_udp.recvfrom(65536)
data, addr

(b'Fake', ('127.0.0.1', 7))

In [6]:
sock_udp.close()

#### TCP socket

In [12]:
sock_tcp = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [13]:
sock_tcp.connect(('127.0.0.1', 13))

In [9]:
sock_tcp.send(b'hello world')

11

In [14]:
data = sock_tcp.recv(1500)

In [15]:
data

b'8:38:54 2018/11/5\n'

In [11]:
sock_tcp.getsockname()

('192.168.44.1', 31643)

In [12]:
sock_tcp.getpeername()

('192.168.44.128', 13)

In [13]:
data.decode("utf-8")

'22 OCT 2018 09:36:11 CST\r\n'

In [16]:
sock_tcp.close()

#### TCP Socket: Server

In [17]:
import socket

In [18]:
host = '127.0.0.1'
port = 5151

In [19]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen(1)
print('Listening at', sock.getsockname())

Listening at ('127.0.0.1', 5151)


In [20]:
sc, sockname = sock.accept()
print('We have accepted a connection from', sockname)

We have accepted a connection from ('127.0.0.1', 12872)


In [15]:
print('  Socket name:', sc.getsockname())
print('  Socket peer:', sc.getpeername())

  Socket name: ('127.0.0.1', 5151)
  Socket peer: ('127.0.0.1', 19679)


In [23]:
mesg = sc.recv(1024)
mesg

b'hello wold'

In [24]:
sc.sendall(mesg)

In [41]:
mesg = sc.recv(1024)
mesg

b'hello'

In [16]:
sc.close()

In [20]:
sock.close()

In [20]:
def sock_recvn(s, length):
    data = b''
    while (len(data) < length):
        data_recv = s.recv(length - len(data))
        if (data_recv == b''):
            break
        data += data_recv
    return data

In [27]:
sock_recvn(sc, 8)

b'orldworl'

#### TCP Ex5_2: Dict Server

In [28]:
endict = dict()

In [29]:
fp = open('新东方红宝书.txt', 'r', encoding='utf8')
for line in fp:
    words = line.split('   ')
    if len(words) == 2:
        key, value = words
        endict[key] = value.replace("\n", "")
fp.close()

In [32]:
english = input('请输入你要查询的单词：')
if english in endict:
    print("%s : %s" % (english, endict[english]))
else:
    print("找不到单词%s" % english)

请输入你要查询的单词：hello
找不到单词hello


In [22]:
import socket

In [35]:
host = '127.0.0.1'
port = 5151

In [36]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen(1)
print('Listening at', sock.getsockname())

Listening at ('127.0.0.1', 5151)


In [55]:
sc, sockname = sock.accept()
print('We have accepted a connection from', sockname)

We have accepted a connection from ('127.0.0.1', 11449)


In [30]:
data = sc.recv(1024)
if data:
    word = data.decode("utf8", errors="ignore")
    if word in endict:
        result = "{} : {}".format(word, endict[word])
    else:
        result = "找不到单词{}".format(word)
    print(word, result)
    sc.sendall(result.encode("utf8"))

abacus abacus : n．算盘



In [41]:
sc.close()

sock_readline1:一个字符一个字符读取

In [51]:
def sock_readline(sock):
    line = b''
    data = b''
    while (data != b'\n'):
        data = sock.recv(1)
        if (data == b''):
            break
        line += data
    return line

In [ ]:
data = sock_readline(sc)

In [59]:
data[:-2]

b'abacus'

In [76]:
data = sock_readline(sc)
if data:
    word = data[:-2].decode("gbk", errors="ignore")
    if word in endict:
        result = "{} : {}".format(word, endict[word])
    else:
        result = "找不到单词{}".format(word)
    print(word, result)
    sc.sendall((result + "\r\n").encode("gbk"))

abacus abacus : n．算盘


In [74]:
word[0:-2]

''

sock_readline2：一次读取一个缓冲区

In [77]:
def sock_readline2(sock, data_buf=b''):
    data_list = []
    index = -1
    data = b''
    if (data_buf != b''):
        data = data_buf
        index = data.find(b'\n')
    
    while (index == -1):
        if (data != b''):
            data_list.append(data)
        
        data = sock.recv(4096)
        if (data == b''):
            break
        
        index = data.find(b'\n')
    
    if (data == b''):
        return (b''.join(data_list), b'')
    
    data_list.append(data[0:index+1])
    return (b''.join(data_list), data[index+1:])

In [ ]:
data_buf = b''

In [84]:
(line, data_buf) = sock_readline2(sc, data_buf)
if line:
    word = line[:-2].decode("gbk", errors="ignore")
    if word in endict:
        result = "{} : {}".format(word, endict[word])
    else:
        result = "找不到单词{}".format(word)
    print(word, result)
    sc.sendall((result + "\r\n").encode("gbk"))

abacus abacus : n．算盘


In [85]:
data_buf

b''

In [34]:
line = b'abacus\r\n'

##### sock_tools

In [2]:
#####################################################
# 常用的socket 读写函数
def sock_readline(sock):
    line = b''
    data = b''
    while (data != b'\n'):
        data = sock.recv(1)
        if (data == b''):
            break
        line += data
    return line

def sock_readn(sock, length):
    data = b''
    while len(data) < length:
        more = sock.recv(length - len(data))
        if not more:
            return data
        data += more
    return data

def sock_writeline(sock, data):
    sock.send(data)
    sock.send(b'\n')

def sock_writen(sock, data, length):
    if (len(data) > length):
        sock.send(data[0:length])
    else:
        sock.send(data)

def sock_write(sock, data):
    sock.send(data)

############################################################
# 提供带buffer缓存的sock读写函数，减少socket函数调用次数，提高效率

def sock_buf_readline(sock, data_buf=b''):
    data_list = []
    index = -1
    data = b''
    if (data_buf != b''):
        data = data_buf
        index = data.find(b'\n')
    
    while (index == -1):
        if (data != b''):
            data_list.append(data)
        
        data = sock.recv(4096)
        if (data == b''):
            break
        
        index = data.find(b'\n')
    
    if (data == b''):
        return (b''.join(data_list), b'')
    
    data_list.append(data[0:index+1])
    return (b''.join(data_list), data[index+1:])

def sock_buf_readn(sock, length, data_buf=b''):
    data_list = []
    
    data = data_buf
    index = len(data_buf)
    while (index < length):
        if (data != b''):
            data_list.append(data)
        
        data = sock.recv(4096)
        if (data == b''):
            break
        
        index += len(data)
    
    if (data == b''):
        return (b''.join(data_list), b'')
    
    index = len(data) - (length - index)
    data_list.append(data[0:index])
    return (b''.join(data_list), data[index:])

def sock_buf_read(sock, data_buf=b''):
    if (len(data_buf) > 0):
        return data_buf
       
    return sock.recv(4096)

#### 定长消息

In [11]:
mesg = "abacus"
mesg = mesg.encode("utf8") 
mesg

b'abacus'

In [14]:
packet = mesg + b' ' * (100 - len(mesg))
packet

b'abacus                                                                                              '

In [15]:
len(packet)

100

In [30]:
PACKET_SIZE = 250

In [25]:
def sendPacket(sock, data):
    sock.sendall(data)
    sock.sendall(b' ' * (PACKET_SIZE - len(data)))

In [26]:
def recvPacket(sock):
    data = sock_readn(sock, PACKET_SIZE)
    if (len(data) != PACKET_SIZE):
        return b''
    data = data.rstrip()
    return data

In [27]:
host = '127.0.0.1'
port = 5151

In [28]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen(1)
print('Listening at', sock.getsockname())

Listening at ('127.0.0.1', 5151)


In [29]:
sc, sockname = sock.accept()
print('We have accepted a connection from', sockname)

We have accepted a connection from ('127.0.0.1', 54575)


In [33]:
recvPacket(sc)

b'hello world'

In [35]:
sendPacket(sc, "farewell!".encode("utf8"))

In [ ]:
sc.close()

In [ ]:
sock.close()

#### 长度前缀--文本长度

In [17]:
mesg = "abacus"
mesg = mesg.encode("utf8") 
mesg

b'abacus'

In [20]:
len_str = "{:05d}".format(len(mesg))
len_str

'00006'

In [21]:
len_str.encode("utf8") + mesg

b'00006abacus'

In [3]:
def sendPacket(sock, data):
    len_str = "{:05d}".format(len(data))
    sock.sendall(len_str.encode("utf8") + data)

In [4]:
def recvPacket(sock):
    data = b''
    len_data = sock_readn(sock, 5)
    if (len(len_data) < 5):
        return data
    length = int(len_data.decode("utf8"))
    data = sock_readn(sock, length)
    return data

In [5]:
host = '127.0.0.1'
port = 5151
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen(1)
print('Listening at', sock.getsockname())

Listening at ('127.0.0.1', 5151)


In [6]:
sc, sockname = sock.accept()
print('We have accepted a connection from', sockname)

We have accepted a connection from ('127.0.0.1', 57794)


In [7]:
recvPacket(sc)

b'hello world'

In [8]:
sendPacket(sc, "farewell!".encode("utf8"))

In [ ]:
sc.close()
sock.close()

#### 长度前缀--二进制长度

In [9]:
hex(4253)

'0x109d'

In [10]:
import struct

In [11]:
#little endian
struct.pack('<i', 4253)

b'\x9d\x10\x00\x00'

In [12]:
#big endian
struct.pack('>i', 4253)

b'\x00\x00\x10\x9d'

In [13]:
struct.unpack('<i', b'\x9d\x10\x00\x00')

(4253,)

In [14]:
struct.unpack('>i', b'\x9d\x10\x00\x00')

(-1659895808,)

#### socket库提供了字节序转换函数 ntohl, ntohs, htonl, htons

In [15]:
#network (= big-endian)
struct.pack('!i', 4253)

b'\x00\x00\x10\x9d'

In [21]:
#network (= big-endian)
struct.unpack('!i', b'\x00\x00\x10\x9d')

(4253,)

In [23]:
length, = struct.unpack('!i', b'\x00\x00\x10\x9d')
length

4253

In [16]:
def sendPacket(sock, data):
    sock.sendall(struct.pack('!h', len(data)) + data)

In [24]:
def recvPacket(sock):
    data = b''
    len_data = sock_readn(sock, 2)
    if (len(len_data) < 2):
        return data
    length, = struct.unpack('!h', len_data)
    data = sock_readn(sock, length)
    return data

In [18]:
host = '127.0.0.1'
port = 5151
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen(1)
print('Listening at', sock.getsockname())

Listening at ('127.0.0.1', 5151)


In [19]:
sc, sockname = sock.accept()
print('We have accepted a connection from', sockname)

We have accepted a connection from ('127.0.0.1', 63814)


In [20]:
recvPacket(sc)

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [25]:
sendPacket(sc, "farewell!".encode("utf8"))

In [26]:
sc.close()
sock.close()

#### “字符”的定义

In [34]:
b'hello'

b'hello'

In [33]:
b'你好'

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-33-234e77bd3686>, line 1)

模式一

In [40]:
address = ("127.0.0.1", 5150)

In [41]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(address)
sock.listen(1)
print('Run this script in another window with "-c" to connect')
print('Listening at', sock.getsockname())
sc, sockname = sock.accept()
print('Accepted connection from', sockname)

Run this script in another window with "-c" to connect
Listening at ('127.0.0.1', 5150)
Accepted connection from ('127.0.0.1', 7148)


In [ ]:
sc.shutdown(socket.SHUT_WR)

In [42]:
message = b''
while True:
    more = sc.recv(8192)  # arbitrary value of 8k
    if not more:  # socket has closed when recv() returns ''
        print('Received zero bytes - end of file')
        break
    print('Received {} bytes'.format(len(more)))
    message += more
print('Message:\n')
print(message.decode('ascii'))

Received 31 bytes
Received 34 bytes
Received 31 bytes
Received zero bytes - end of file
Message:

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.



In [43]:
sc.close()
sock.close()

#### pickle格式

In [44]:
import pickle

In [45]:
pickle.dumps([5, 6, 7]) 

b'\x80\x03]q\x00(K\x05K\x06K\x07e.'

In [48]:
pickle.loads(b'\x80\x03]q\x00(K\x05K\x06K\x07e.')

[5, 6, 7]

In [50]:
pickle.loads(b'\x80\x03]q\x00(K\x05K\x06K\x07e.blahblahblah')

[5, 6, 7]

In [49]:
pickle.loads(b'\x80\x03]q\x00')

EOFError: Ran out of input

In [51]:
from io import BytesIO

In [52]:
f = BytesIO(b'\x80\x03]q\x00(K\x05K\x06K\x07e.blahblahblah')

In [53]:
pickle.load(f)

[5, 6, 7]

In [54]:
f.tell()

14

In [55]:
f.read()

b'blahblahblah'

In [56]:
type(f)

_io.BytesIO

In [68]:
address = ("127.0.0.1", 5150)

In [69]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(address)
sock.listen(1)
print('Run this script in another window with "-c" to connect')
print('Listening at', sock.getsockname())
sc, sockname = sock.accept()
print('Accepted connection from', sockname)

Run this script in another window with "-c" to connect
Listening at ('127.0.0.1', 5150)
Accepted connection from ('127.0.0.1', 11189)


In [70]:
sc_file = sc.makefile(mode='rb')

In [71]:
type(sc_file)

_io.BufferedReader

In [75]:
pickle.load(sc_file)

[5, 6, 7]

In [76]:
pickle.load(sc_file)

{'age': '18', 'score': [80, 98, 68], 'username': 'zhangsan'}

In [77]:
sc.close()
sock.close()

#### 压缩

In [78]:
import zlib

In [79]:
data = zlib.compress(b'Python') + b'.' + zlib.compress(b'zlib') + b'.'

In [80]:
data

b'x\x9c\x0b\xa8,\xc9\xc8\xcf\x03\x00\x08\x97\x02\x83.x\x9c\xab\xca\xc9L\x02\x00\x04d\x01\xb2.'

In [81]:
len(data)

28

In [84]:
zlib.decompress(data)

b'Python'

In [82]:
d = zlib.decompressobj()

In [83]:
d.decompress(data[0:8]), d.unused_data

(b'Pytho', b'')

In [85]:
d.decompress(data[8:16]), d.unused_data

(b'n', b'.x')

In [86]:
d = zlib.decompressobj()

In [87]:
d.decompress(b'x'), d.unused_data

(b'', b'')

In [88]:
d.decompress(data[16:24]), d.unused_data

(b'zlib', b'')

In [89]:
d.decompress(data[24:]), d.unused_data

(b'', b'.')

#### DNS

In [90]:
socket.getaddrinfo("www.sina.com.cn", "www")

[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  0,
  '',
  ('183.232.24.222', 80)),
 (<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  0,
  '',
  ('112.25.53.229', 80))]

In [91]:
socket.getaddrinfo("www.sina.com.cn", "www", flags=socket.AI_CANONNAME)

[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  0,
  'spool.grid.sinaedge.com',
  ('183.232.24.222', 80)),
 (<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  0,
  '',
  ('112.25.53.229', 80))]

In [92]:
socket.gethostname()

'DESKTOP-MQ5KL6Q'

In [93]:
socket.getfqdn()

'DESKTOP-MQ5KL6Q'

In [95]:
socket.gethostbyname("www.sina.com.cn")

'183.232.24.222'

In [96]:
socket.gethostbyaddr('183.232.24.222')

herror: [Errno 11004] host not found

In [97]:
socket.gethostbyaddr('210.32.32.1')

herror: [Errno 11004] host not found

In [99]:
socket.getservbyname("www")

80

In [100]:
socket.getservbyname("http")

80

In [98]:
socket.getservbyport(80)

'http'